# Unsupervised Learning (Clustering Algorithm)

In [31]:
!uv pip install \
  "fastparquet>=2025.12.0" \
  "ipykernel>=7.2.0" \
  "llvmlite>=0.42" \
  "matplotlib>=3.10.8" \
  "numba>=0.59" \
  "numpy>=2.3.5" \
  "pandas==2.3.3" \
  "pandas-stubs==2.3.3.260113" \
  "prophet>=1.3.0" \
  "scikit-learn>=1.8.0" \
  "scikit-learn>=1.8.0" \
  "scipy>=1.17.0" \
  "seaborn>=0.13.2" \
  "umap-learn>=0.5.11"

Using Python 3.12.12 environment at: /Users/z.yang/playground/srh-stat-and-ml-exam/.venv
Audited 14 packages in 33ms


In [25]:
from pathlib import Path

ROOT_DIR = Path().resolve().parent
DATA_DIR = ROOT_DIR / "data"

In [26]:
import pandas as pd

pd.set_option("future.no_silent_downcasting", True)
pd.set_option('display.max_columns', None)

In [27]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [32]:
# ----------------------------------------
# UMAP + DBSCAN cluster
# ----------------------------------------
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

import umap


def _build_feature_matrix(df: pd.DataFrame) -> pd.DataFrame:
    X = df.drop(columns=[c for c in ["id", "date"] if c in df.columns]).copy()

    num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
    bool_cols = [c for c in X.columns if pd.api.types.is_bool_dtype(X[c])]
    X = X[num_cols + bool_cols].copy()

    # bool -> 0/1 for distance-based methods
    for c in bool_cols:
        X[c] = X[c].astype(int)

    return X


def run_umap_dbscan(
    df: pd.DataFrame,
    # preprocessing
    fill_value: float = 0.0,
    # umap
    umap_dim_for_clustering: int = 10,
    umap_dim_for_plot: int = 2,
    n_neighbors: int = 30,
    min_dist: float = 0.0,
    metric: str = "euclidean",
    random_state: int = 42,
    # dbscan
    eps: float = 0.6,
    min_samples: int = 10,
):
    """
    Returns:
      df_out: df + cluster label (-1 noise)
      Zc: UMAP embedding used for clustering (n x umap_dim_for_clustering)
      Z2: UMAP 2D embedding for plotting (n x 2)
      counts: cluster size counts (including -1)
      sil: silhouette on non-noise points in Zc (if >=2 clusters)
    """
    df_out = df.copy()
    X = _build_feature_matrix(df_out)

    prep = Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value=fill_value)),
        ("scaler", StandardScaler()),
    ])
    Xs = prep.fit_transform(X)

    reducer_cluster = umap.UMAP(
        n_components=umap_dim_for_clustering,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric=metric,
        random_state=random_state,
    )
    Zc = reducer_cluster.fit_transform(Xs)

    reducer_plot = umap.UMAP(
        n_components=umap_dim_for_plot,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric=metric,
        random_state=random_state,
    )
    Z2 = reducer_plot.fit_transform(Xs)

    db = DBSCAN(eps=eps, min_samples=min_samples, metric="euclidean")
    labels = db.fit_predict(Zc)

    df_out["cluster"] = labels

    counts = pd.Series(labels).value_counts().sort_index()
    noise_frac = float((labels == -1).mean())
    n_clusters = int(len([k for k in counts.index if k != -1]))

    sil = np.nan
    mask = labels != -1
    if mask.sum() > 10:
        uniq = np.unique(labels[mask])
        if uniq.size >= 2:
            sil = silhouette_score(Zc[mask], labels[mask])

    print(f"DBSCAN clusters (excluding noise): {n_clusters}")
    print(f"Noise fraction: {noise_frac:.3f}")
    print("Cluster counts:\n", counts)

    return df_out, Zc, Z2, counts, sil

In [34]:
# ----------------------------------------
# Search for best parameter `eps`
# ----------------------------------------
def scan_eps(
    df: pd.DataFrame,
    eps_list=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0),
    min_samples: int = 10,
    umap_dim_for_clustering: int = 10,
    n_neighbors: int = 30,
    min_dist: float = 0.0,
    random_state: int = 42,
):
    rows = []
    for eps in eps_list:
        df_out, Zc, Z2, counts, sil = run_umap_dbscan(
            df,
            umap_dim_for_clustering=umap_dim_for_clustering,
            n_neighbors=n_neighbors,
            min_dist=min_dist,
            random_state=random_state,
            eps=eps,
            min_samples=min_samples,
        )
        n_clusters = (counts.index != -1).sum()
        noise_frac = float((df_out["cluster"] == -1).mean())
        rows.append({"eps": eps, "min_samples": min_samples, "n_clusters": int(n_clusters), "noise_frac": noise_frac, "silhouette": sil})
    return pd.DataFrame(rows).sort_values(["n_clusters", "noise_frac"])


daily_df = pd.read_parquet(DATA_DIR / "processed" / "unsupervised" / "diabetes_daily_features.parquet")
grid = scan_eps(
    daily_df,
    eps_list=[0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    min_samples=10)
display(grid)

DBSCAN clusters (excluding noise): 19
Noise fraction: 0.010
Cluster counts:
 -1      33
 0     631
 1     265
 2     356
 3     132
 4      60
 5     469
 6      51
 7      54
 8      35
 9     185
 10    108
 11    150
 12     34
 13     76
 14    113
 15    362
 16     66
 17    133
 18     65
Name: count, dtype: int64
DBSCAN clusters (excluding noise): 16
Noise fraction: 0.002
Cluster counts:
 -1       7
 0     907
 1     887
 2     134
 3      60
 4      51
 5     150
 6      36
 7     187
 8     110
 9      34
 10     76
 11    113
 12    362
 13     66
 14    133
 15     65
Name: count, dtype: int64
DBSCAN clusters (excluding noise): 12
Noise fraction: 0.000
Cluster counts:
 -1        1
 0      907
 1     1252
 2      137
 3       60
 4       51
 5      216
 6       36
 7       34
 8       76
 9      113
 10     362
 11     133
Name: count, dtype: int64
DBSCAN clusters (excluding noise): 12
Noise fraction: 0.000
Cluster counts:
 0      907
1     1252
2      138
3       60
4      

,eps,min_samples,n_clusters,noise_frac,silhouette
6,0.9,10,6,0.000000,0.430593
7,1.0,10,6,0.000000,0.430593
5,0.8,10,7,0.000000,0.394616
4,0.7,10,8,0.000000,0.398662
3,0.6,10,12,0.000000,0.257102
2,0.5,10,12,0.000296,0.257353
1,0.4,10,16,0.002072,0.334640
0,0.3,10,19,0.009769,0.445536


In [36]:
# ----------------------------------------
# Get result by the best parameters
# ----------------------------------------
df_db, Zc, Z2, counts, sil = run_umap_dbscan(
    daily_df,
    eps=1.0,
    min_samples=10,
)

print("Silhouette:", sil)
print("Cluster counts:")
print(counts)

DBSCAN clusters (excluding noise): 6
Noise fraction: 0.000
Cluster counts:
 0    1321
1    1288
2      51
3     110
4     113
5     495
Name: count, dtype: int64
Silhouette: 0.43059346079826355
Cluster counts:
0    1321
1    1288
2      51
3     110
4     113
5     495
Name: count, dtype: int64


In [39]:
# ----------------------------------------
# Calculate the features for each cluster
# ----------------------------------------
key_features = [
    # glycemic level & variability
    "bg_all_mean",
    "bg_all_std",
    "bg_all_range",

    # treatment intensity
    "ins_all_sum",
    "ins_all_count",

    # monitoring / activity
    "bg_all_count",
    "n_events",
    "active_span_minutes",

    # events / risk indicators
    "meal_events_count",
    "exercise_events_count",
    "hypo_flag",
]

key_features = [c for c in key_features if c in df_db.columns]

cluster_summary = (
    df_db
    .groupby("cluster")[key_features]
    .mean(numeric_only=True)
    .sort_index()
)

cluster_sizes = df_db["cluster"].value_counts().sort_index()
cluster_summary.insert(0, "cluster_size", cluster_sizes.values)
cluster_summary_rounded = cluster_summary.round(3)
cluster_summary_rounded

,cluster_size,bg_all_mean,bg_all_std,bg_all_range,ins_all_sum,ins_all_count,bg_all_count,n_events,active_span_minutes,meal_events_count,exercise_events_count,hypo_flag
cluster,,,,,,,,,,,,
0,1321,157.827,45.228,90.793,35.323,3.338,3.241,6.642,715.333,0.030,0.004,0.018
1,1288,155.187,69.691,138.519,29.599,3.765,3.311,7.636,829.113,0.226,0.047,0.158
2,51,208.082,95.808,214.216,38.843,4.706,4.294,9.353,898.549,0.118,0.059,0.059
3,110,173.849,76.457,162.227,28.209,4.127,3.782,9.955,845.445,0.709,0.918,0.191
4,113,126.664,60.101,129.504,21.752,3.062,3.903,8.690,840.575,0.159,1.115,0.133
5,495,172.942,65.083,143.879,46.329,4.248,3.949,8.305,911.737,0.079,0.000,0.028


In [40]:
# ----------------------------------------
# A heuristic for interpreting the risk level of each cluster
# ----------------------------------------
risk_score = (
    cluster_summary["bg_all_std"]
    + 0.5 * cluster_summary["bg_all_range"]
    + 0.2 * cluster_summary["ins_all_sum"]
    + 5.0 * cluster_summary["hypo_flag"]
)

cluster_summary["risk_score"] = risk_score
cluster_summary.sort_values("risk_score", ascending=False)

,cluster_size,bg_all_mean,bg_all_std,bg_all_range,ins_all_sum,ins_all_count,bg_all_count,n_events,active_span_minutes,meal_events_count,exercise_events_count,hypo_flag,risk_score
cluster,,,,,,,,,,,,,
2,51,208.081839,95.808054,214.215686,38.843137,4.705882,4.294118,9.352941,898.549020,0.117647,0.058824,0.058824,210.978643
3,110,173.848636,76.456929,162.227273,28.209091,4.127273,3.781818,9.954545,845.445455,0.709091,0.918182,0.190909,164.166929
5,495,172.942222,65.083197,143.878788,46.329293,4.248485,3.949495,8.305051,911.737374,0.078788,0.000000,0.028283,146.429864
1,1288,155.186511,69.690567,138.518634,29.599379,3.764752,3.310559,7.635870,829.112578,0.225932,0.047360,0.157609,145.657803
4,113,126.663717,60.101444,129.504425,21.752212,3.061947,3.902655,8.690265,840.575221,0.159292,1.115044,0.132743,129.867816
0,1321,157.826729,45.227615,90.793338,35.322861,3.338380,3.240727,6.641938,715.333081,0.030280,0.003785,0.018168,97.779697


## Conclusion

| Cluster <br /> (Desc by Risk Level) | Name                                   | Interpretation                                                                                                                                                 |
|------------------------------------:|----------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------|
|                                   2 | Severely unstable, high-risk days      | Extremely high glucose levels and variability with frequent corrective interventions, representing days of severe instability and elevated clinical risk.      |
|                                   3 | Behavior-driven high-variability days  | Pronounced glucose fluctuations strongly associated with meal and exercise events, often accompanied by hypoglycemic symptoms.                                 |
|                                   5 | Correction-heavy hyperglycemic days    | Persistently elevated glucose levels with intensive insulin usage, indicating frequent corrective dosing rather than acute instability.                        |
|                                   1 | Moderately unstable routine days       | Typical daily patterns with moderate glucose variability and occasional risk indicators, reflecting routine but imperfect glycemic control.                    |
|                                   4 | Exercise-associated lower-glucose days | Lower average glucose levels combined with frequent exercise events, suggesting a physically active regime with moderate variability.                          |
|                                   0 | Stable, well-controlled days           | Low glycemic variability and narrow glucose range with few events and limited intervention, indicating relatively stable and well-controlled daily management. |
